In [2]:
### csv 파일 읽어오기 코드 작성. 
import pandas as pd
import os
import numpy as np

C:\Users\admin\AppData\Local\Temp\ipykernel_46864\3657941114.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [57]:
data = []

output_folder = f"../../../../../MultiModal_postprocessing/results/face/face_landmark/"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for group in ["A", "B", "C", "D", "E", "F", "G"]:
    os.chdir(f"D:/MultiModal/MultiModal_postprocessing/results/face/face_landmark_postprocessing/{group}/")
    #os.chdir(f"D:/MultiModal/MultiModal_postprocessing/results/face_5m/face_landmark/{group}/")

    step_list = ["S1", "S2", "S3"]
    #step_list = ["S1_1", "S1_2", "S1_3", "S1_4", "S2_1", "S2_2", "S2_3", "S2_4", "S3"]

    face_index = ["Group", "Week",] + [f"{step}_Face_lean_3p" for step in step_list]

    for week in [f"{w_index}W" for w_index in range(1, 5)]:
        step_data = []
        for step in step_list:
            df1 = pd.read_csv(f'Face_{week}_{group}1_{step}.csv') if os.path.exists(f'Face_{week}_{group}1_{step}.csv') else []
            df2 = pd.read_csv(f'Face_{week}_{group}2_{step}.csv') if os.path.exists(f'Face_{week}_{group}2_{step}.csv') else []
            df3 = pd.read_csv(f'Face_{week}_{group}3_{step}.csv') if os.path.exists(f'Face_{week}_{group}3_{step}.csv') else []
            df4 = pd.read_csv(f'Face_{week}_{group}4_{step}.csv') if os.path.exists(f'Face_{week}_{group}4_{step}.csv') else []
                        
            a = np.array([ len(df1) if len(df1) > 0 else 1000000000000,len(df2) if len(df2) > 0 else 1000000000000, len(df3) if len(df3) > 0 else 1000000000000,len(df4) if len(df4) > 0 else 1000000000000])
            length = a.min()-1

            def plus_minus(df):
                df["box size"] = df['box.width'] * df['box.height']

                W = "box.width"
                H = "box.height"


                df["box delta"] = df["box size"].diff()
                df_new = df.iloc[1:,18]

                for i in range(len(df_new)):
                    if(df_new.iloc[i] >600):
                        df_new.iloc[i] = 1
                    elif(df_new.iloc[i] <-600):
                        df_new.iloc[i] = -1
                    else:
                        df_new.iloc[i] =0
                return df_new

            
            if len(df1) > 0:
                df1 = plus_minus(df1).iloc[:length]
            else:
                df1 = pd.DataFrame({"box delta": [0 for _ in range(length)]})
            
            if len(df2) > 0:
                df2 = plus_minus(df2).iloc[:length]
            else:
                df2 = pd.DataFrame({"box delta": [0 for _ in range(length)]})

            if len(df3) > 0:
                df3 = plus_minus(df3).iloc[:length]
            else:
                df3 = pd.DataFrame({"box delta": [0 for _ in range(length)]})

            if len(df4) > 0:
                df4 = plus_minus(df4).iloc[:length]
            else:
                df4 = pd.DataFrame({"box delta": [0 for _ in range(length)]})
            
            df_concat = pd.concat([df1,df2,df3,df4], axis = 1)

            df_concat['same'] = 0 #garbage
            for i in range (len(df_concat)):
                
                count=df_concat.iloc[i].value_counts()
                try:
                    count[0] = 0
                except:
                    0
            
                df_concat.iloc[i,4] = count.max()
            same = np.array(df_concat["same"])
            step_data.append(np.where(same>=3,1,0).sum())
        data.append([group, week] + step_data)

In [ ]:
count_df = pd.DataFrame(data, columns=face_index)

In [59]:
count_df.to_csv(output_folder + "All_Face_lean_count.csv")

In [61]:
os.chdir(f"D:/MultiModal/MultiModal_postprocessing/results/face/face_landmark/")
all_df = pd.read_csv("All_Face_lean_count.csv") 

step_list = ["S1", "S2", "S3"]
#step_list = ["S1_1", "S1_2", "S1_3", "S1_4", "S2_1", "S2_2", "S2_3", "S2_4", "S3"]

df_index = ["Group"] + [f"{week}_{step}_Face_lean_3p" for week in ["1W", "2W", "3W", "4W"] for step in step_list] 

data = []
for group in ["A", "B", "C", "D", "E", "F", "G"]:
    week_data = []
    for week in [f"{w_index}W" for w_index in range(1, 5)]:
        filtered_df = all_df[(all_df["Week"] == week) & (all_df["Group"] == group)]
        week_data.extend(filtered_df[[f"{step}_Face_lean_3p" for step in step_list]].values.flatten().tolist())
    
    data.append([group] + week_data)   

all_summarize_df = pd.DataFrame(data, columns=df_index)
all_summarize_df.to_excel("All_Face_lean_count(All group, excel_anaysis).xlsx")